Using the images marked as valid per cluster, we pass them through the CNN and extract their feature vectors. the results are stored at a per-country basis. For example, all Malawi feature extractions will go into results/malawi_2016/cnn.

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [15]:
BASE_DIR = '..'
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'cnn_images')
CNN_DIR = os.path.join(BASE_DIR, 'models', 'trained_model.pt')

In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(RESULTS_DIR, country), exist_ok=True)

# Feature extract with CNN
If you have run this step before, you can skip it and run the commented out code in the next section to quick-start.

In [6]:
df_images = pd.read_csv(os.path.join(PROCESSED_DIR, 'image_download_actual.csv'))

In [7]:
df_images.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin,is_train
0,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0,True
1,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.09515,35.217213,1.423239,0.025206,mw,0,True
2,-17.125093842803985_35.18726915719602_-17.0951...,-17.125094,35.187269,-17.09515,35.217213,1.423239,0.025206,mw,0,False
3,-17.140065764205975_35.20224107859801_-17.0951...,-17.140066,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0,True
4,-17.065206157196016_35.20224107859801_-17.0951...,-17.065206,35.202241,-17.09515,35.217213,1.423239,0.025206,mw,0,True


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} as backend')
model = torch.load(CNN_DIR, map_location=device)

Using cuda as backend


In [17]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)

In [18]:
# rip off the final layers
model.classifier = model.classifier[:4]

In [19]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [33]:
transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# custom dataset for fast image loading and processing
# does not follow the usual style of folder -> folder for each class -> image
# we just want one folder with images
class ForwardPassDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transformer):
        self.image_dir = image_dir
        self.image_list = os.listdir(self.image_dir)
        self.transformer = transformer

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image_name = self.image_list[index]

        # Load image
        X = self.filename_to_im_tensor(self.image_dir + '/' + image_name)
        
        # dataloaders need to return a label, but for the forward pass we don't really care
        return X, -1
    
    def filename_to_im_tensor(self, file):
        im = plt.imread(file)[:,:,:3]
        im = self.transformer(im)
        return im

model.eval()  
classes = [0, 1, 2]
# shape of final array will be (num_validation_images, 4096)
# we also want to record the image each index represents
feats = np.zeros(((~df_images['is_train']).sum(), 4096))
image_order = []
i = 0
for c in classes:
    # use the validation images to do the forward pass
    dataset = ForwardPassDataset(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(c)), transformer)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
    image_order += dataset.image_list
    # forward pass for this class
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        feats[i:i+len(inputs),:] = outputs.cpu().detach().numpy()
        i += len(inputs)

In [46]:
feats

array([[-1.26415765, -0.28140783, -0.29993755, ...,  0.33739716,
        -0.96456331, -0.95310527],
       [ 0.55027246, -0.06091447,  0.11403629, ..., -0.08996978,
        -0.62236136, -0.96085918],
       [ 0.52193987, -0.29220241, -0.45371717, ...,  0.34175205,
        -1.1439786 , -0.85960728],
       ...,
       [-0.50936353,  0.39209121, -0.29870456, ...,  0.0661362 ,
         0.43009469, -0.34069228],
       [ 0.24428365,  0.07818466, -0.89307284, ...,  0.29522306,
        -0.72958505, -1.24356151],
       [-0.30123377,  0.6785413 , -0.19940855, ...,  0.14395328,
         0.52420121, -1.16047859]])

In [36]:
forward_pass_df = pd.DataFrame.from_dict({'image_name': image_order, 'feat_index': np.arange(len(image_order))})
forward_pass_df.head()

,image_name,feat_index
0,10.513181862198008_39.768191057994024_10.52815...,0
1,-14.632534157196016_34.981995235794024_-14.662...,1
2,-14.526346764205977_35.593520078598004_-14.481...,2
3,7.4290196173360155_7.26950147266_7.45896346014...,3
4,-10.405547698244352_34.14279535535209_-10.4038...,4


In [37]:
df_consumption = pd.merge(left=df_images, right=forward_pass_df, on='image_name')

In [40]:
# have we maintained all validation images?
assert len(df_consumption) == (~df_images['is_train']).sum()

In [41]:
df_consumption.head()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,country,nightlights_bin,is_train,feat_index
0,-17.125093842803985_35.18726915719602_-17.0951...,-17.125094,35.187269,-17.095150,35.217213,1.423239,0.025206,mw,0,False,318
1,-17.140065764205975_35.232184921401995_-17.095...,-17.140066,35.232185,-17.095150,35.217213,1.423239,0.025206,mw,0,False,1861
2,-17.065206157196016_35.262128764205976_-17.095...,-17.065206,35.262129,-17.095150,35.217213,1.423239,0.025206,mw,0,False,836
3,-17.07737907859801_35.069727235794026_-17.0923...,-17.077379,35.069727,-17.092351,35.114643,1.266204,0.000000,mw,0,False,18
4,-17.137266764205975_35.08469915719602_-17.0923...,-17.137267,35.084699,-17.092351,35.114643,1.266204,0.000000,mw,0,False,1051


## Aggregate Features
For each country, we aggregate the image features per cluster and save them to results/country/cnn

In [47]:
country_abbrv = ['mw', 'eth', 'ng']
country_dir = ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']

for ca, cd in zip(country_abbrv, country_dir):
    df_c = df_consumption[df_consumption['country'] == ca]
    group = df_c.groupby(['cluster_lat', 'cluster_lon'])
    x = np.zeros((len(group), 4096))
    cluster_list = [] # the corresponding clusters (lat, lon) to the x aggregate feature array
    for i, g in enumerate(group):
        lat, lon = g[0]
        im_sub = df_consumption[(df_consumption['cluster_lat'] == lat) & (df_consumption['cluster_lon'] == lon)].reset_index(drop=True)
        agg_feats = np.zeros((len(im_sub), 4096))
        for j, d in im_sub.iterrows():
            agg_feats[j,:] = feats[d.feat_index]
        agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster

        x[i,:] = agg_feats
        cluster_list.append([lat, lon])
    # save to the correct directory
    save_dir = os.path.join(RESULTS_DIR, cd, 'cnn')
    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, 'cluster_feats.npy'), x)
    pickle.dump(cluster_list, open(os.path.join(save_dir, 'cluster_order.pkl'), 'wb')) 
    